* Pobrać z kaglehub wybrany dataset
* wykorzystać pandas i obliczyć poznane statystyki opisowe kolumn numerycznych
* Wygenerować 4 wykresy najlepiej opisujące dany dataset i do każdego dołączyć komentarz opisujący dlaczego dany wykres pasuje do użytych danych.
* Napisać podsumowanie, w któym należy podać pomysł wykorzystania wybranego datasetu.

In [2]:
import pandas as pd
import numpy as np
import matplotlib as mat
import plotly as pl
import kagglehub
from kagglehub import KaggleDatasetAdapter
import json
import os

ModuleNotFoundError: No module named 'plotly'